In [ ]:
import os
import pandas as pd

directory = "/Users/vincewu/Bees-Project"
log_folder = "Data/DPC (Compiled)/"
processed_folder = "Fully Processed"
tag_id_file = 'tag_ids.csv'

def hour_to_time(original_num):
    seconds = original_num*3600
    hours = int(seconds/3600)
    seconds -= hours*3600
    minutes = int(seconds/60)
    seconds -= minutes*60
    seconds = int(seconds)

    hours_str = str(hours)
    if(len(hours_str) == 1):
        hours_str = '0'+hours_str
    minutes_str = str(minutes)
    if(len(minutes_str) == 1):
        minutes_str = '0'+minutes_str
    seconds_str = str(seconds)
    if(len(seconds_str) == 1):
        seconds_str = '0'+seconds_str

    return hours_str+":"+minutes_str+":"+seconds_str


activity = []
overall_day = 0

tag_id_csv = pd.read_csv(os.path.join(
    directory, processed_folder, tag_id_file), engine='python', header=None)
tag_ids = {}
for i in range(len(tag_id_csv)):
    if tag_id_csv.loc[i,1]!=None:
        tag_ids[tag_id_csv.loc[i,1]] = tag_id_csv.iloc[i,0]

filelist = os.listdir(os.path.join(directory, log_folder))
for filename in sorted(filelist):
    if filename.endswith('.csv'):
        print(filename)
        pathname = os.path.join(directory, log_folder, filename)

        file_length = len(filename)
        # if filename[file_length-6:file_length-5] == ' ':
        #     subtract = 16
        # elif filename[file_length-8:file_length-4] == 'copy':
        #     subtract = 19
        # else:
        #     subtract = 14
        date_temp = []
        date_num = 0
        date_index = 0
        while date_index < file_length:
            if date_num >= 3:
                break
            if filename[date_index] >= '0' and filename[date_index] <= '9':
                for j in range(date_index+1, file_length):
                    if filename[j] < '0' or filename[j] > '9':
                        date_temp.append(filename[date_index:j])
                        date_index = j
                        date_num += 1
                        break
            date_index += 1

        year = int(date_temp[0])
        month = int(date_temp[1])
        day = int(date_temp[2])
        # curr_date = ['', '']
        # curr_date[0] = '-'.join([date_temp[0], date_temp[1], date_temp[2]])
        # curr_date[1] = '/'.join([str(month), str(day), date_temp[0][2:]])
        curr_date = '-'.join([date_temp[0], date_temp[1], date_temp[2]])

        # year = int(filename[file_length-(subtract+10)                   :file_length-(subtract+6)])
        # month = int(filename[file_length-(subtract+5)                    :file_length-(subtract+3)])
        # day = int(filename[file_length-(subtract+2):file_length-subtract])

        # if month<start_month:
        #     start_month=month
        #     start_day=day
        # elif month==start_month and day<start_day:
        #     start_day=day

        # if month>end_month:
        #     end_month=month
        #     end_day=day
        # elif month==end_month and day>end_day:
        #     end_day=day

        # file_count[month][day]-=1

        try:
            df = pd.read_csv(pathname, engine='python',
                             sep="[;,. ]", header=None)
        except pd.errors.EmptyDataError:
            df = pd.DataFrame()

        if not df.empty:
            overall_day += 1
            activity.append({})
            print(curr_date)

            df.dropna(axis=1, how='all', inplace=True)
            # df.columns=[i for i in range(len(df.columns))]

            # df=df.iloc[:, [0, 1, 5, 14]]
            df.columns = ['Date', 'Timestamp', 'UID', 'Direction']

            # df_end=len(df)-1
            # for i in range(len(df['Date'])):
            #     if df['Date'][i]!=curr_date[0] and df['Date'][i]!=curr_date[1]:
            #         print(df['Date'][i], "index "+str(i))
            #         df_end=i-1
            #         overflow=pd.concat([overflow, df[df_end+1:len(df)]], axis=0, ignore_index=True)
            #         df=df.drop(df.index[df_end+1:len(df)])
            #         break

            # df=pd.concat([addon, df], axis=0, ignore_index=True)

            ids = len(df["UID"])
            keys = set()

            for i in range(ids):
                key = df["UID"][i]
                if not key in keys:
                    keys.add(key)

                    activity[-1][key] = {"Day": overall_day, "Date": curr_date, "Group": "x",
                                         "Trip Durations": [], "Trip Durations (Hours)": [],
                                         "Trip Amount": 0, "Trip Average Duration": 0,
                                         "Trip Average Duration (Hours)": 0, "Rest Durations": [],
                                         "Rest Durations (Hours)": [], "Rest Amount": 0, "Rest Average Duration": 0,
                                         "Rest Average Duration (Hours)": 0, "First Detection": 0,
                                         "First Detection (Hours)": 0, "Last Detection": 0,
                                         "Last Detection (Hours)": 0, "Trip Start and End Times": [],
                                         "Trip Start and End Times (Hours)": [], "All Start Times": [],
                                         "All Start Times (Hours)": [], "All End Times": [], "All End Times (Hours)": [],
                                         "1 Minute or Less Trips": [], "1 Minute or Less Trips (Hours)": [], "1 Minute Amount": 0,
                                         "1 Minute Average Duration": 0, "1 Minute Average Duration (Hours)": 0,
                                         "1 Minute Rest Durations": [],  "1 Minute Rest Durations (Hours)": [],
                                         "1 Minute Rest Amount": 0, "1 Minute Rest Average Duration": 0,
                                         "1 Minute Rest Average Duration (Hours)": 0, "1-3 Minute Trips": [],
                                         "1-3 Minute Trips (Hours)": [], "1-3 Minute Amount": 0, "1-3 Minute Average Duration": 0,
                                         "1-3 Minute Average Duration (Hours)": 0,
                                         "1-3 Minute Rest Durations": [],  "1-3 Minute Rest Durations (Hours)": [],
                                         "1-3 Minute Rest Amount": 0, "1-3 Minute Rest Average Duration": 0,
                                         "1-3 Minute Rest Average Duration (Hours)": 0,
                                         "3-6 Minute Trips": [], "3-6 Minute Trips (Hours)": [],
                                         "3-6 Minute Amount": 0, "3-6 Minute Average Duration": 0,
                                         "3-6 Minute Average Duration (Hours)": 0,
                                         "3-6 Minute Rest Durations": [],  "3-6 Minute Rest Durations (Hours)": [],
                                         "3-6 Minute Rest Amount": 0, "3-6 Minute Rest Average Duration": 0,
                                         "3-6 Minute Rest Average Duration (Hours)": 0, "6 Minute or More Trips": [],
                                         "6 Minute or More Trips (Hours)": [], "6 Minute Amount": 0,
                                         "6 Minute Average Duration": 0, "6 Minute Average Duration (Hours)": 0,
                                         "6 Minute Rest Durations": [],  "6 Minute Rest Durations (Hours)": [],
                                         "6 Minute Rest Amount": 0, "6 Minute Rest Average Duration": 0,
                                         "6 Minute Rest Average Duration (Hours)": 0}

                    # activity[-1][key]["Date"] = '/'.join(
                    #     [str(month), str(day), str(year)])

                    #activity[-1][key]=[int(), "x", [], [], int(), int(), [], [], int(), [], int(), [], int(), [], int()]
                    if key in tag_ids:
                        activity[-1][key]["Group"] = tag_ids[key]
                    else:
                        activity[-1][key]["Group"] = "No Group"
                    sames = []

                    prev_time = -1
                    add = 0.001
                    for j in range(i, ids):
                        if df["UID"][j] == key:
                            if df["Direction"][j] == 'Unknown':
                                continue
                            temp = df["Timestamp"][j]
                            #time_length=len(temp)

                            time_temp = []
                            ti = 0
                            for k in range(len(temp)):
                                if temp[k] == ':':
                                    time_temp.append(temp[ti:k])
                                    ti = k+1
                                elif k == len(temp)-1:
                                    time_temp.append(temp[ti:])

                            hour = int(time_temp[0])
                            minute = int(time_temp[1])
                            second = int(float(time_temp[2]))
                            # hour = int(temp[:time_length-6])
                            # minute = int(temp[time_length-5:time_length-3])
                            # second = int(float(temp[time_length-2:]))
                            # time = hour*3600+minute*60+second
                            time = hour+minute/60+second/3600
                            #print(time)
                            if time == prev_time:
                                time += add
                                add += 0.00000001
                            else:
                                prev_time = time
                                add = 0.00000001

                            #print(temp, time, sep=" ")

                            sames.append([time, df["Direction"][j]])

                    if len(sames) == 0:
                        continue
                    first_key = True

                    # sames.sort()
                    #print(sames)

                    index = 0
                    # secure_index = -1
                    activity[-1][key]["First Detection"] = hour_to_time(
                        sames[0][0])
                    activity[-1][key]["First Detection (Hours)"] = sames[0][0]
                    activity[-1][key]["Last Detection"] = hour_to_time(
                        sames[-1][0])
                    activity[-1][key]["Last Detection (Hours)"] = sames[-1][0]

                    prev_time = -1
                    index = 0

                    # print(key)

                    while index < len(sames)-1 and sames[index][1] == "Arriving":
                        index += 1
                    # latest "Departing", earliest "Arriving"
                    while index < len(sames):
                        while index < len(sames)-1 and sames[index+1][1] == "Departing":
                            index += 1
                        # ends on a series of "Departing"
                        if index == len(sames)-1:
                            break

                        # print(index, index+1, sep=' ')

                        start_time = sames[index][0]
                        end_time=sames[index+1][0]

                        # if end_time-start_time>=0.05:
                        if prev_time != -1:
                            activity[-1][key]["Rest Average Duration (Hours)"] += start_time - \
                                prev_time
                            activity[-1][key]["Rest Amount"] += 1
                            activity[-1][key]["Rest Durations"].append(
                                hour_to_time((start_time-prev_time)))
                            activity[-1][key]["Rest Durations (Hours)"].append(
                                (start_time-prev_time))

                        activity[-1][key]["Trip Start and End Times"].append(
                            [hour_to_time(start_time)])
                        activity[-1][key]["Trip Start and End Times (Hours)"].append([
                            start_time])
                        activity[-1][key]["All Start Times"].append(
                            hour_to_time(start_time))
                        activity[-1][key]["All Start Times (Hours)"].append(
                            start_time)

                        index += 1

                        activity[-1][key]["Trip Average Duration (Hours)"] += end_time - \
                            start_time
                        activity[-1][key]["Trip Amount"] += 1
                        activity[-1][key]["Trip Durations"].append(
                            hour_to_time(end_time-start_time))
                        activity[-1][key]["Trip Durations (Hours)"].append(
                            end_time-start_time)

                        # activity[-1][key]["Last Detection"]=hour_to_time(end_time)
                        # activity[-1][key]["Last Detection (Hours)"]=end_time
                        activity[-1][key]["All End Times"].append(
                            hour_to_time(end_time))
                        activity[-1][key]["All End Times (Hours)"].append(
                            end_time)
                        activity[-1][key]["Trip Start and End Times"][-1].append(
                            hour_to_time(end_time))
                        activity[-1][key]["Trip Start and End Times (Hours)"][-1].append(
                            end_time)
                        prev_time = end_time

                        if end_time-start_time <= 60/3600:
                            if activity[-1][key]["1 Minute Amount"] >= 1:
                                rest_duration = start_time - \
                                    activity[-1][key]["1 Minute or Less Trips (Hours)"][-1][1]
                                activity[-1][key]["1 Minute Rest Durations"].append(
                                    hour_to_time(rest_duration))
                                activity[-1][key]["1 Minute Rest Durations (Hours)"].append(
                                    rest_duration)
                                activity[-1][key]["1 Minute Rest Amount"] += 1
                                activity[-1][key]["1 Minute Rest Average Duration (Hours)"] += rest_duration

                            activity[-1][key]["1 Minute or Less Trips"].append(
                                [hour_to_time(start_time), hour_to_time(end_time)])
                            activity[-1][key]["1 Minute or Less Trips (Hours)"].append([
                                start_time, end_time])
                            activity[-1][key]["1 Minute Amount"] += 1
                            activity[-1][key]["1 Minute Average Duration (Hours)"] += end_time - \
                                start_time
                        elif end_time-start_time <= 180/3600:
                            if activity[-1][key]["1-3 Minute Amount"] >= 1:
                                rest_duration = start_time-activity[-1][key][
                                    "1-3 Minute Trips (Hours)"][-1][1]
                                activity[-1][key]["1-3 Minute Rest Durations"].append(
                                    hour_to_time(rest_duration))
                                activity[-1][key]["1-3 Minute Rest Durations (Hours)"].append(
                                    rest_duration)
                                activity[-1][key]["1-3 Minute Rest Amount"] += 1
                                activity[-1][key]["1-3 Minute Rest Average Duration (Hours)"] += rest_duration
                            activity[-1][key]["1-3 Minute Trips"].append(
                                [hour_to_time(start_time), hour_to_time(end_time)])
                            activity[-1][key]["1-3 Minute Trips (Hours)"].append([
                                start_time, end_time])
                            activity[-1][key]["1-3 Minute Amount"] += 1
                            activity[-1][key]["1-3 Minute Average Duration (Hours)"] += end_time - \
                                start_time
                        elif end_time-start_time <= 360/3600:
                            if activity[-1][key]["3-6 Minute Amount"] >= 1:
                                rest_duration = start_time - \
                                    activity[-1][key]["3-6 Minute Trips (Hours)"][-1][1]
                                activity[-1][key]["3-6 Minute Rest Durations"].append(
                                    hour_to_time(rest_duration))
                                activity[-1][key]["3-6 Minute Rest Durations (Hours)"].append(
                                    rest_duration)
                                activity[-1][key]["3-6 Minute Rest Amount"] += 1
                                activity[-1][key]["3-6 Minute Rest Average Duration (Hours)"] += rest_duration
                            activity[-1][key]["3-6 Minute Trips"].append(
                                [hour_to_time(start_time), hour_to_time(end_time)])
                            activity[-1][key]["3-6 Minute Trips (Hours)"].append([
                                start_time, end_time])
                            activity[-1][key]["3-6 Minute Amount"] += 1
                            activity[-1][key]["3-6 Minute Average Duration (Hours)"] += end_time - \
                                start_time
                        else:
                            if activity[-1][key]["6 Minute Amount"] >= 1:
                                rest_duration = start_time - \
                                    activity[-1][key]["6 Minute or More Trips (Hours)"][-1][1]
                                activity[-1][key]["6 Minute Rest Durations"].append(
                                    hour_to_time(rest_duration))
                                activity[-1][key]["6 Minute Rest Durations (Hours)"].append(
                                    rest_duration)
                                activity[-1][key]["6 Minute Rest Amount"] += 1
                                activity[-1][key]["6 Minute Rest Average Duration (Hours)"] += rest_duration
                            activity[-1][key]["6 Minute or More Trips"].append(
                                [hour_to_time(start_time), hour_to_time(end_time)])
                            activity[-1][key]["6 Minute or More Trips (Hours)"].append([
                                start_time, end_time])
                            activity[-1][key]["6 Minute Amount"] += 1
                            activity[-1][key]["6 Minute Average Duration (Hours)"] += end_time - \
                                start_time
                    
                        # else:
                        #     index+=1

                        while index < len(sames)-1 and sames[index+1][1] == "Arriving":
                            index += 1

        # for key in activity[month][day]:
        #     print(key, ": ", activity[-1][key])

        # if file_count[month][day]==0:
                    activity[-1][key]["Rest Average Duration"] = hour_to_time(
                        activity[-1][key]["Rest Average Duration (Hours)"])

                    if activity[-1][key]["Trip Amount"] > 0:
                        activity[-1][key]["Trip Average Duration (Hours)"] /= activity[-1][key]["Trip Amount"]
                        activity[-1][key]["Trip Average Duration"] = hour_to_time(
                            activity[-1][key]["Trip Average Duration (Hours)"])
                    if activity[-1][key]["Rest Amount"] > 0:
                        activity[-1][key]["Rest Average Duration (Hours)"] /= activity[-1][key]["Rest Amount"]
                        activity[-1][key]["Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["Rest Average Duration (Hours)"])

                    if activity[-1][key]["1 Minute Amount"] > 0:
                        activity[-1][key]["1 Minute Average Duration (Hours)"] /= activity[-1][key]["1 Minute Amount"]
                        activity[-1][key]["1 Minute Average Duration"] = hour_to_time(
                            activity[-1][key]["1 Minute Average Duration (Hours)"])
                    if activity[-1][key]["1 Minute Rest Amount"] > 0:
                        activity[-1][key]["1 Minute Rest Average Duration (Hours)"] /= activity[-1][key]["1 Minute Rest Amount"]
                        activity[-1][key]["1 Minute Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["1 Minute Rest Average Duration (Hours)"])

                    if activity[-1][key]["1-3 Minute Amount"] > 0:
                        activity[-1][key]["1-3 Minute Average Duration (Hours)"] /= activity[-1][key]["1-3 Minute Amount"]
                        activity[-1][key]["1-3 Minute Average Duration"] = hour_to_time(
                            activity[-1][key]["1-3 Minute Average Duration (Hours)"])
                    if activity[-1][key]["1-3 Minute Rest Amount"] > 0:
                        activity[-1][key]["1-3 Minute Rest Average Duration (Hours)"] /= activity[-1][
                            key]["1-3 Minute Rest Amount"]
                        activity[-1][key]["1-3 Minute Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["1-3 Minute Rest Average Duration (Hours)"])

                    if activity[-1][key]["3-6 Minute Amount"] > 0:
                        activity[-1][key]["3-6 Minute Average Duration (Hours)"] /= activity[-1][key]["3-6 Minute Amount"]
                        activity[-1][key]["3-6 Minute Average Duration"] = hour_to_time(
                            activity[-1][key]["3-6 Minute Average Duration (Hours)"])
                    if activity[-1][key]["3-6 Minute Rest Amount"] > 0:
                        activity[-1][key]["3-6 Minute Rest Average Duration (Hours)"] /= activity[-1][
                            key]["3-6 Minute Rest Amount"]
                        activity[-1][key]["3-6 Minute Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["3-6 Minute Rest Average Duration (Hours)"])

                    if activity[-1][key]["6 Minute Amount"] > 0:
                        activity[-1][key]["6 Minute Average Duration (Hours)"] /= activity[-1][key]["6 Minute Amount"]
                        activity[-1][key]["6 Minute Average Duration"] = hour_to_time(
                            activity[-1][key]["6 Minute Average Duration (Hours)"])
                    if activity[-1][key]["6 Minute Rest Amount"] > 0:
                        activity[-1][key]["6 Minute Rest Average Duration (Hours)"] /= activity[-1][key]["6 Minute Rest Amount"]
                        activity[-1][key]["6 Minute Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["6 Minute Rest Average Duration (Hours)"])
                        
        # ts
        # break

In [ ]:
#CHECKPOINT TEST

processed_folder = "Fully Processed"

final_df = []
for i in activity:
    final_df.append(pd.DataFrame.from_dict(i, orient="index"))

column_headers = ["Day", "Date", "Group", "Trip Durations", "Trip Durations (Hours)",
                  "Trip Amount", "Trip Average Duration",
                  "Trip Average Duration (Hours)", "Rest Durations",
                  "Rest Durations (Hours)", "Rest Amount", "Rest Average Duration",
                  "Rest Average Duration (Hours)", "First Detection",
                  "First Detection (Hours)", "Last Detection",
                  "Last Detection (Hours)", "Trip Start and End Times",
                  "Trip Start and End Times (Hours)", "All Start Times",
                  "All Start Times (Hours)", "All End Times", "All End Times (Hours)",
                  "1 Minute or Less Trips", "1 Minute or Less Trips (Hours)", "1 Minute Amount",
                  "1 Minute Average Duration", "1 Minute Average Duration (Hours)",
                  "1 Minute Rest Durations",  "1 Minute Rest Durations (Hours)",
                  "1 Minute Rest Amount", "1 Minute Rest Average Duration",
                  "1 Minute Rest Average Duration (Hours)", "1-3 Minute Trips",
                  "1-3 Minute Trips (Hours)", "1-3 Minute Amount", "1-3 Minute Average Duration",
                  "1-3 Minute Average Duration (Hours)",
                  "1-3 Minute Rest Durations",  "1-3 Minute Rest Durations (Hours)",
                  "1-3 Minute Rest Amount", "1-3 Minute Rest Average Duration",
                  "1-3 Minute Rest Average Duration (Hours)",
                  "3-6 Minute Trips", "3-6 Minute Trips (Hours)",
                  "3-6 Minute Amount", "3-6 Minute Average Duration",
                  "3-6 Minute Average Duration (Hours)", "6 Minute or More Trips",
                  "3-6 Minute Rest Durations",  "3-6 Minute Rest Durations (Hours)",
                  "3-6 Minute Rest Amount", "3-6 Minute Rest Average Duration",
                  "3-6 Minute Rest Average Duration (Hours)",
                  "6 Minute or More Trips (Hours)", "6 Minute Amount",
                  "6 Minute Average Duration", "6 Minute Average Duration (Hours)",
                  "6 Minute Rest Durations",  "6 Minute Rest Durations (Hours)",
                  "6 Minute Rest Amount", "6 Minute Rest Average Duration",
                  "6 Minute Rest Average Duration (Hours)"]
# print(start_month, start_day, end_month, end_day, day, sep=" ")

final_df_csv = pd.concat(final_df)

final_df_csv.to_csv(directory+'/'+processed_folder +
                    '/processed_orientation.csv', header=column_headers)


In [ ]:
len(activity)

In [ ]:
import os
import pandas as pd

directory = "/Users/vincewu/Bees-Project"
processed_folder = "Fully Processed"
tag_id_file = 'tag_ids.csv'

log_folder = "Data/DOC (Compiled)"
# tag_id_file = 'tag_ids.csv'

# tag_id_csv = pd.read_csv(os.path.join(
#     directory, tag_id_file), engine='python', header=None)
# tag_ids = {}
# for i in range(len(tag_id_csv[0])):
#     tag_ids[tag_id_csv.loc[i][1]] = tag_id_csv.loc[i][0]

filelist = os.listdir(os.path.join(directory, log_folder))
for filename in sorted(filelist):
    if filename.endswith('.csv'):
        print(filename)
        pathname = os.path.join(directory, log_folder, filename)

        file_length = len(filename)
        # if filename[file_length-6:file_length-5] == ' ':
        #     subtract = 16
        # elif filename[file_length-8:file_length-4] == 'copy':
        #     subtract = 19
        # else:
        #     subtract = 14
        date_temp = []
        date_num = 0
        date_index = 0
        while date_index < file_length:
            if date_num >= 3:
                break
            if filename[date_index] >= '0' and filename[date_index] <= '9':
                for j in range(date_index+1, file_length):
                    if filename[j] < '0' or filename[j] > '9':
                        date_temp.append(filename[date_index:j])
                        date_index = j
                        date_num += 1
                        break
            date_index += 1

        year = int(date_temp[0])
        month = int(date_temp[1])
        day = int(date_temp[2])
        curr_date = '-'.join([date_temp[0], date_temp[1], date_temp[2]])

        # year = int(filename[file_length-(subtract+10)                   :file_length-(subtract+6)])
        # month = int(filename[file_length-(subtract+5)                    :file_length-(subtract+3)])
        # day = int(filename[file_length-(subtract+2):file_length-subtract])

        # if month<start_month:
        #     start_month=month
        #     start_day=day
        # elif month==start_month and day<start_day:
        #     start_day=day

        # if month>end_month:
        #     end_month=month
        #     end_day=day
        # elif month==end_month and day>end_day:
        #     end_day=day

        # file_count[month][day]-=1

        try:
            df = pd.read_csv(pathname, engine='python',
                             sep="[;,. ]", header=None)
        except pd.errors.EmptyDataError:
            df = pd.DataFrame()

        if not df.empty:
            overall_day += 1
            activity.append({})
            print(curr_date)
            # data_days.append([month, day, year])

            df.dropna(axis=1, how='all', inplace=True)
            # df.columns=[i for i in range(len(df.columns))]

            # df=df.iloc[:, [0, 1, 5, 7]]
            df.columns=['Date', 'Timestamp', 'UID', 'Ant. Number']

            # df_end=len(df)-1
            # for i in range(len(df['Date'])):
            #     if df['Date'][i]!=curr_date:
            #         df_end=i-1
            #         overflow=pd.concat([overflow, df[df_end+1:len(df)]], axis=0, ignore_index=True)
            #         df=df.drop(df.index[df_end+1:len(df)])
            #         break
            
            # df=pd.concat([addon, df], axis=0, ignore_index=True)

            # print(df)

            # temp_columns = df.iloc[0].tolist()
            # df.loc[-1] = temp_columns
            # df.index = df.index+1
            # df = df.sort_index()

            # time_found = False
            # for i in range(len(temp_columns)):
            #     column = temp_columns[i]
            #     col_length = len(column)

            #     #temp_length=len(column)
            #     if column[2:3] == ":" and not time_found:
            #         temp_columns[i] = "Timestamp"
            #         time_found = True
            #     elif column[:4] == "25SQ":
            #         temp_columns[i] = "UID"
            #     elif column == "1" or column == "2":
            #         temp_columns[i] = "Ant. Number"

            # df.columns = temp_columns

            # for column in df.columns:
            #     if column != "Timestamp" and column != "UID" and column != "Ant. Number":
            #         del(df[column])

            # print(df)

            ids = len(df["UID"])
            keys = set()

            for i in range(ids):
                key = df["UID"][i]
                if not key in keys:
                    keys.add(key)

                    activity[-1][key] = {"Day": overall_day, "Date": curr_date, "Group": "x",
                                         "Trip Durations": [], "Trip Durations (Hours)": [],
                                         "Trip Amount": 0, "Trip Average Duration": 0,
                                         "Trip Average Duration (Hours)": 0, "Rest Durations": [],
                                         "Rest Durations (Hours)": [], "Rest Amount": 0, "Rest Average Duration": 0,
                                         "Rest Average Duration (Hours)": 0, "First Detection": 0,
                                         "First Detection (Hours)": 0, "Last Detection": 0,
                                         "Last Detection (Hours)": 0, "Trip Start and End Times": [],
                                         "Trip Start and End Times (Hours)": [], "All Start Times": [],
                                         "All Start Times (Hours)": [], "All End Times": [], "All End Times (Hours)": [],
                                         "1 Minute or Less Trips": [], "1 Minute or Less Trips (Hours)": [], "1 Minute Amount": 0,
                                         "1 Minute Average Duration": 0, "1 Minute Average Duration (Hours)": 0,
                                         "1 Minute Rest Durations": [],  "1 Minute Rest Durations (Hours)": [],
                                         "1 Minute Rest Amount": 0, "1 Minute Rest Average Duration": 0,
                                         "1 Minute Rest Average Duration (Hours)": 0, "1-3 Minute Trips": [],
                                         "1-3 Minute Trips (Hours)": [], "1-3 Minute Amount": 0, "1-3 Minute Average Duration": 0,
                                         "1-3 Minute Average Duration (Hours)": 0,
                                         "1-3 Minute Rest Durations": [],  "1-3 Minute Rest Durations (Hours)": [],
                                         "1-3 Minute Rest Amount": 0, "1-3 Minute Rest Average Duration": 0,
                                         "1-3 Minute Rest Average Duration (Hours)": 0,
                                         "3-6 Minute Trips": [], "3-6 Minute Trips (Hours)": [],
                                         "3-6 Minute Amount": 0, "3-6 Minute Average Duration": 0,
                                         "3-6 Minute Average Duration (Hours)": 0,
                                         "3-6 Minute Rest Durations": [],  "3-6 Minute Rest Durations (Hours)": [],
                                         "3-6 Minute Rest Amount": 0, "3-6 Minute Rest Average Duration": 0,
                                         "3-6 Minute Rest Average Duration (Hours)": 0, "6 Minute or More Trips": [],
                                         "6 Minute or More Trips (Hours)": [], "6 Minute Amount": 0,
                                         "6 Minute Average Duration": 0, "6 Minute Average Duration (Hours)": 0,
                                         "6 Minute Rest Durations": [],  "6 Minute Rest Durations (Hours)": [],
                                         "6 Minute Rest Amount": 0, "6 Minute Rest Average Duration": 0,
                                         "6 Minute Rest Average Duration (Hours)": 0}

                    # activity[-1][key]["Date"] = '/'.join(
                    #     [str(month), str(day), str(year)])

                    #activity[-1][key]=[int(), "x", [], [], int(), int(), [], [], int(), [], int(), [], int(), [], int()]
                    if key in tag_ids:
                        activity[-1][key]["Group"] = tag_ids[key]
                    else:
                        activity[-1][key]["Group"] = "No Group"
                    sames_doc = []

                    prev_time = -1
                    add = 0.001
                    for j in range(i, ids):
                        if df["UID"][j] == key:
                            temp = df["Timestamp"][j]
                            #time_length=len(temp)

                            time_temp = []
                            ti = 0
                            for k in range(len(temp)):
                                if temp[k] ==':':
                                    time_temp.append(temp[ti:k])
                                    ti=k+1
                                elif k==len(temp)-1:
                                    time_temp.append(temp[ti:])
                                        
                            hour = int(time_temp[0])
                            minute = int(time_temp[1])
                            second = int(float(time_temp[2]))
                                        
                            hour = int(time_temp[0])
                            minute = int(time_temp[1])
                            second = int(float(time_temp[2]))
                            # hour = int(temp[:time_length-6])
                            # minute = int(temp[time_length-5:time_length-3])
                            # second = int(float(temp[time_length-2:]))
                            # time = hour*3600+minute*60+second
                            time = hour+minute/60+second/3600
                            #print(time)
                            if time == prev_time:
                                time += add
                                add += 0.00000001
                            else:
                                prev_time = time
                                add = 0.00000001

                            #print(temp, time, sep=" ")

                            sames_doc.append([time, int(df["Ant. Number"][j])])

                    first_key = True

                    # sames.sort()
                    #print(sames)
                    sames=[]

                    index = 0
                    secure_index = -1
                    activity[-1][key]["First Detection"] = hour_to_time(
                        sames_doc[0][0])
                    activity[-1][key]["First Detection (Hours)"] = sames_doc[0][0]
                    activity[-1][key]["Last Detection"] = hour_to_time(
                        sames_doc[-1][0])
                    activity[-1][key]["Last Detection (Hours)"] = sames_doc[-1][0]

                    #try 2-1-1-2 only

                    while index < len(sames_doc)-1:
                        if sames_doc[index][1] == 2 and sames_doc[index+1][1] == 1:
                            sames.append([sames_doc[index][0], 'Departing'])
                            index+=1
                        elif sames_doc[index][1] == 1 and sames_doc[index+1][1] == 2:
                            sames.append([sames_doc[index][0], 'Arriving'])
                            index+=1
                        index+=1
                            

                    #     first_time = sames[index][0]
                    #     time_groups.append([])
                    #     reader_groups.append([])
                    #     directions.append("Unknown")
                    #     time_groups[-1].append(sames[index][0])
                    #     reader_groups[-1].append(sames[index][1])

                    #     index += 1
                    #     while index < len(sames):
                    #         # seconds in terms of hours
                    #         if sames[index][0]-30/3600 > first_time:
                    #             break
                    #         time_groups[-1].append(sames[index][0])
                    #         reader_groups[-1].append(sames[index][1])
                    #         index += 1

                    #     if len(reader_groups[-1]) >= 2:
                    #         if reader_groups[-1][0] == 1 and reader_groups[-1][-1] == 2:
                    #             directions[-1] = "Arriving"
                    #         elif reader_groups[-1][0] == 2 and reader_groups[-1][-1] == 1:
                    #             directions[-1] = "Departing"

                    #         if secure_index == -1 and (directions[-1] == "Arriving" or directions[-1] == "Departing"):
                    #             secure_index = len(directions)-1

                    # time_groups = [[float(i) for i in block]
                    #                for block in time_groups]
                    # #print(time_groups)
                    # #found a secure arrival or departure
                    # if secure_index != -1:
                    #     #use parity to assign value based on distance from secure index
                    #     possible_directions = [directions[secure_index], "x"]
                    #     if directions[secure_index] == "Arriving":
                    #         possible_directions[1] = "Departing"
                    #     else:
                    #         possible_directions[1] = "Arriving"

                    #     for i in range(len(directions)):
                    #         if directions[i] == "Unknown":
                    #             distance = abs(i-secure_index) % 2
                    #             directions[i] = possible_directions[distance]

                    #if no secure arrival or departure, everything is unknown
                    #didn't happen a single time in the sample

                    prev_time = -1
                    index = 0
                    while index < len(sames)-1 and sames[index][1] == "Arriving":
                        index += 1
                    # latest "Departing", earliest "Arriving"
                    while index < len(sames):
                        while index < len(sames)-1 and sames[index+1][1] == "Departing":
                            index += 1
                        # ends on a series of "Departing"
                        if index == len(sames)-1:
                            break

                        # print(index, index+1, sep=' ')

                        start_time = sames[index][0]
                        end_time = sames[index+1][0]

                        # if end_time-start_time >= 0.05:
                        if prev_time != -1:
                            activity[-1][key]["Rest Average Duration (Hours)"] += start_time - \
                                prev_time
                            activity[-1][key]["Rest Amount"] += 1
                            activity[-1][key]["Rest Durations"].append(
                                hour_to_time((start_time-prev_time)))
                            activity[-1][key]["Rest Durations (Hours)"].append(
                                (start_time-prev_time))

                        activity[-1][key]["Trip Start and End Times"].append(
                            [hour_to_time(start_time)])
                        activity[-1][key]["Trip Start and End Times (Hours)"].append([
                            start_time])
                        activity[-1][key]["All Start Times"].append(
                            hour_to_time(start_time))
                        activity[-1][key]["All Start Times (Hours)"].append(
                            start_time)

                        index += 1

                        activity[-1][key]["Trip Average Duration (Hours)"] += end_time - \
                            start_time
                        activity[-1][key]["Trip Amount"] += 1
                        activity[-1][key]["Trip Durations"].append(
                            hour_to_time(end_time-start_time))
                        activity[-1][key]["Trip Durations (Hours)"].append(
                            end_time-start_time)

                        # activity[-1][key]["Last Detection"]=hour_to_time(end_time)
                        # activity[-1][key]["Last Detection (Hours)"]=end_time
                        activity[-1][key]["All End Times"].append(
                            hour_to_time(end_time))
                        activity[-1][key]["All End Times (Hours)"].append(
                            end_time)
                        activity[-1][key]["Trip Start and End Times"][-1].append(
                            hour_to_time(end_time))
                        activity[-1][key]["Trip Start and End Times (Hours)"][-1].append(
                            end_time)
                        prev_time = end_time

                        if end_time-start_time <= 60/3600:
                            if activity[-1][key]["1 Minute Amount"] >= 1:
                                rest_duration = start_time - \
                                    activity[-1][key]["1 Minute or Less Trips (Hours)"][-1][1]
                                activity[-1][key]["1 Minute Rest Durations"].append(
                                    hour_to_time(rest_duration))
                                activity[-1][key]["1 Minute Rest Durations (Hours)"].append(
                                    rest_duration)
                                activity[-1][key]["1 Minute Rest Amount"] += 1
                                activity[-1][key]["1 Minute Rest Average Duration (Hours)"] += rest_duration

                            activity[-1][key]["1 Minute or Less Trips"].append(
                                [hour_to_time(start_time), hour_to_time(end_time)])
                            activity[-1][key]["1 Minute or Less Trips (Hours)"].append([
                                start_time, end_time])
                            activity[-1][key]["1 Minute Amount"] += 1
                            activity[-1][key]["1 Minute Average Duration (Hours)"] += end_time - \
                                start_time
                        elif end_time-start_time <= 180/3600:
                            if activity[-1][key]["1-3 Minute Amount"] >= 1:
                                rest_duration = start_time-activity[-1][key][
                                    "1-3 Minute Trips (Hours)"][-1][1]
                                activity[-1][key]["1-3 Minute Rest Durations"].append(
                                    hour_to_time(rest_duration))
                                activity[-1][key]["1-3 Minute Rest Durations (Hours)"].append(
                                    rest_duration)
                                activity[-1][key]["1-3 Minute Rest Amount"] += 1
                                activity[-1][key]["1-3 Minute Rest Average Duration (Hours)"] += rest_duration
                            activity[-1][key]["1-3 Minute Trips"].append(
                                [hour_to_time(start_time), hour_to_time(end_time)])
                            activity[-1][key]["1-3 Minute Trips (Hours)"].append([
                                start_time, end_time])
                            activity[-1][key]["1-3 Minute Amount"] += 1
                            activity[-1][key]["1-3 Minute Average Duration (Hours)"] += end_time - \
                                start_time
                        elif end_time-start_time <= 360/3600:
                            if activity[-1][key]["3-6 Minute Amount"] >= 1:
                                rest_duration = start_time - \
                                    activity[-1][key]["3-6 Minute Trips (Hours)"][-1][1]
                                activity[-1][key]["3-6 Minute Rest Durations"].append(
                                    hour_to_time(rest_duration))
                                activity[-1][key]["3-6 Minute Rest Durations (Hours)"].append(
                                    rest_duration)
                                activity[-1][key]["3-6 Minute Rest Amount"] += 1
                                activity[-1][key]["3-6 Minute Rest Average Duration (Hours)"] += rest_duration
                            activity[-1][key]["3-6 Minute Trips"].append(
                                [hour_to_time(start_time), hour_to_time(end_time)])
                            activity[-1][key]["3-6 Minute Trips (Hours)"].append([
                                start_time, end_time])
                            activity[-1][key]["3-6 Minute Amount"] += 1
                            activity[-1][key]["3-6 Minute Average Duration (Hours)"] += end_time - \
                                start_time
                        else:
                            if activity[-1][key]["6 Minute Amount"] >= 1:
                                rest_duration = start_time - \
                                    activity[-1][key]["6 Minute or More Trips (Hours)"][-1][1]
                                activity[-1][key]["6 Minute Rest Durations"].append(
                                    hour_to_time(rest_duration))
                                activity[-1][key]["6 Minute Rest Durations (Hours)"].append(
                                    rest_duration)
                                activity[-1][key]["6 Minute Rest Amount"] += 1
                                activity[-1][key]["6 Minute Rest Average Duration (Hours)"] += rest_duration
                            activity[-1][key]["6 Minute or More Trips"].append(
                                [hour_to_time(start_time), hour_to_time(end_time)])
                            activity[-1][key]["6 Minute or More Trips (Hours)"].append([
                                start_time, end_time])
                            activity[-1][key]["6 Minute Amount"] += 1
                            activity[-1][key]["6 Minute Average Duration (Hours)"] += end_time - \
                                start_time

                        # else:
                        #     index += 1

                        while index < len(sames)-1 and sames[index+1][1] == "Arriving":
                            index += 1

        # for key in activity[month][day]:
        #     print(key, ": ", activity[-1][key])

        # if file_count[month][day]==0:
                    activity[-1][key]["Rest Average Duration"] = hour_to_time(
                        activity[-1][key]["Rest Average Duration (Hours)"])

                    if activity[-1][key]["Trip Amount"] > 0:
                        activity[-1][key]["Trip Average Duration (Hours)"] /= activity[-1][key]["Trip Amount"]
                        activity[-1][key]["Trip Average Duration"] = hour_to_time(
                            activity[-1][key]["Trip Average Duration (Hours)"])
                    if activity[-1][key]["Rest Amount"] > 0:
                        activity[-1][key]["Rest Average Duration (Hours)"] /= activity[-1][key]["Rest Amount"]
                        activity[-1][key]["Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["Rest Average Duration (Hours)"])

                    if activity[-1][key]["1 Minute Amount"] > 0:
                        activity[-1][key]["1 Minute Average Duration (Hours)"] /= activity[-1][key]["1 Minute Amount"]
                        activity[-1][key]["1 Minute Average Duration"] = hour_to_time(
                            activity[-1][key]["1 Minute Average Duration (Hours)"])
                    if activity[-1][key]["1 Minute Rest Amount"] > 0:
                        activity[-1][key]["1 Minute Rest Average Duration (Hours)"] /= activity[-1][key]["1 Minute Rest Amount"]
                        activity[-1][key]["1 Minute Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["1 Minute Rest Average Duration (Hours)"])

                    if activity[-1][key]["1-3 Minute Amount"] > 0:
                        activity[-1][key]["1-3 Minute Average Duration (Hours)"] /= activity[-1][key]["1-3 Minute Amount"]
                        activity[-1][key]["1-3 Minute Average Duration"] = hour_to_time(
                            activity[-1][key]["1-3 Minute Average Duration (Hours)"])
                    if activity[-1][key]["1-3 Minute Rest Amount"] > 0:
                        activity[-1][key]["1-3 Minute Rest Average Duration (Hours)"] /= activity[-1][
                            key]["1-3 Minute Rest Amount"]
                        activity[-1][key]["1-3 Minute Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["1-3 Minute Rest Average Duration (Hours)"])

                    if activity[-1][key]["3-6 Minute Amount"] > 0:
                        activity[-1][key]["3-6 Minute Average Duration (Hours)"] /= activity[-1][key]["3-6 Minute Amount"]
                        activity[-1][key]["3-6 Minute Average Duration"] = hour_to_time(
                            activity[-1][key]["3-6 Minute Average Duration (Hours)"])
                    if activity[-1][key]["3-6 Minute Rest Amount"] > 0:
                        activity[-1][key]["3-6 Minute Rest Average Duration (Hours)"] /= activity[-1][
                            key]["3-6 Minute Rest Amount"]
                        activity[-1][key]["3-6 Minute Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["3-6 Minute Rest Average Duration (Hours)"])

                    if activity[-1][key]["6 Minute Amount"] > 0:
                        activity[-1][key]["6 Minute Average Duration (Hours)"] /= activity[-1][key]["6 Minute Amount"]
                        activity[-1][key]["6 Minute Average Duration"] = hour_to_time(
                            activity[-1][key]["6 Minute Average Duration (Hours)"])
                    if activity[-1][key]["6 Minute Rest Amount"] > 0:
                        activity[-1][key]["6 Minute Rest Average Duration (Hours)"] /= activity[-1][key]["6 Minute Rest Amount"]
                        activity[-1][key]["6 Minute Rest Average Duration"] = hour_to_time(
                            activity[-1][key]["6 Minute Rest Average Duration (Hours)"])



In [ ]:
processed_folder = "Fully Processed"

final_df = []
for i in activity:
    final_df.append(pd.DataFrame.from_dict(i, orient="index"))

column_headers = ["Day", "Date", "Group", "Trip Durations", "Trip Durations (Hours)",
                  "Trip Amount", "Trip Average Duration",
                  "Trip Average Duration (Hours)", "Rest Durations",
                  "Rest Durations (Hours)", "Rest Amount", "Rest Average Duration",
                  "Rest Average Duration (Hours)", "First Detection",
                  "First Detection (Hours)", "Last Detection",
                  "Last Detection (Hours)", "Trip Start and End Times",
                  "Trip Start and End Times (Hours)", "All Start Times",
                  "All Start Times (Hours)", "All End Times", "All End Times (Hours)",
                  "1 Minute or Less Trips", "1 Minute or Less Trips (Hours)", "1 Minute Amount",
                  "1 Minute Average Duration", "1 Minute Average Duration (Hours)",
                  "1 Minute Rest Durations",  "1 Minute Rest Durations (Hours)",
                  "1 Minute Rest Amount", "1 Minute Rest Average Duration",
                  "1 Minute Rest Average Duration (Hours)", "1-3 Minute Trips",
                  "1-3 Minute Trips (Hours)", "1-3 Minute Amount", "1-3 Minute Average Duration",
                  "1-3 Minute Average Duration (Hours)",
                  "1-3 Minute Rest Durations",  "1-3 Minute Rest Durations (Hours)",
                  "1-3 Minute Rest Amount", "1-3 Minute Rest Average Duration",
                  "1-3 Minute Rest Average Duration (Hours)",
                  "3-6 Minute Trips", "3-6 Minute Trips (Hours)",
                  "3-6 Minute Amount", "3-6 Minute Average Duration",
                  "3-6 Minute Average Duration (Hours)", "6 Minute or More Trips",
                  "3-6 Minute Rest Durations",  "3-6 Minute Rest Durations (Hours)",
                  "3-6 Minute Rest Amount", "3-6 Minute Rest Average Duration",
                  "3-6 Minute Rest Average Duration (Hours)",
                  "6 Minute or More Trips (Hours)", "6 Minute Amount",
                  "6 Minute Average Duration", "6 Minute Average Duration (Hours)",
                  "6 Minute Rest Durations",  "6 Minute Rest Durations (Hours)",
                  "6 Minute Rest Amount", "6 Minute Rest Average Duration",
                  "6 Minute Rest Average Duration (Hours)"]
# print(start_month, start_day, end_month, end_day, day, sep=" ")

final_df_csv = pd.concat(final_df)

final_df_csv.to_csv(directory+'/'+processed_folder +
                    '/processed_orientation.csv', header=column_headers)
